In [1]:
# https://medium.com/@erikhallstrm/hello-world-rnn-83cd7105b767
DEBUG = True

import re, random, math, csv, io, string, itertools
import numpy as np
import pandas as pd
import tensorflow as tf

In [2]:
hp = dict(
    n_layers = 2,
    hidden_size = 512,
    fc_size = 512,
    dropout = 0.9,
    batch_size = 20,
    lr = 0.001,
    lr_decay = 0.9999,
    min_lr = 0.00001,
    grad_clip = 5.,
    cuda = False,
    num_epoch = 5,
    max_length = 10
)

In [3]:
class Voc:
    SOS = "!"
    EOS = "#"
    SOS_ID = 0
    EOS_ID = 1
    def __init__(self):
        self.word2index = {self.SOS:0, self.EOS:1}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2 # Count SOS and EOS

    def index_words(self, sentence):
        for word in sentence.split(' '):
            self.index_word(word)

    def index_word(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

voc = Voc()

for c in itertools.chain(range(ord('a'), ord('z')+1),range(ord('A'),ord('Z')+1)):
    voc.index_word(chr(c))
print(f'vocabulary size: {voc.n_words}')

vocabulary size: 54


In [4]:
class PairGenerator:
#     vocabulary = [chr(i) for i in itertools.chain(range(ord('a'), ord('z')+1),range(ord('A'),ord('Z')+1))]
    word_len_interval = {"a":2,"b":7}
    sent_len_interval = {"a":1,"b":10}
    
    def __init__(self,voc):
        self.voc = voc
        self.vocabulary = [c for c in voc.word2index.keys() if c not in {voc.SOS, voc.EOS}]
    def gen_word_pair(self):
        word_len = int(random.uniform(**self.word_len_interval))
        word = random.choices(self.vocabulary,k=word_len)
        return "".join(word), "".join(list(reversed(word)))
    
    def gen_pair(self):
        num_words = int(random.uniform(**self.sent_len_interval))
        inp, out = zip(*[self.gen_word_pair() for _ in range(num_words)])
        return self.voc.SOS+" ".join(inp)+self.voc.EOS, self.voc.SOS+" ".join(out)+self.voc.EOS
    
    def gen_batch(self, n_unrollings):
        inp, out = zip(*[self.gen_pair() for _ in range(n_unrollings)])
        return inp, out
        
pg = PairGenerator(voc)

In [ ]:
class RNN:
    pass